<a href="https://colab.research.google.com/github/arikmitra/SMS_Text_Classifier/blob/main/SMS_Text_Classifier_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2024-01-17 09:15:18--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2024-01-17 09:15:19 (29.2 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-01-17 09:15:19--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

In [ ]:
train_df = pd.read_csv(
                            'train-data.tsv',sep='\t',
                            header = None,
                            names=['class', 'text']
                            )
val_df = pd.read_csv(
                            'valid-data.tsv',
                             sep='\t',
                             header = None,
                             names=['class', 'text']
                             )

In [ ]:
#train_df.head()

In [ ]:
train_df['class'].replace(['ham', 'spam'],[0,1], inplace=True)
val_df['class'].replace(['ham', 'spam'],[0,1], inplace=True)

In [ ]:
#train_df.head()

In [ ]:
train_labels = train_df["class"].values
val_labels = val_df["class"].values

In [ ]:
train_tensor = tf.data.Dataset.from_tensor_slices((train_df['text'].values, train_labels)).batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
val_tensor = tf.data.Dataset.from_tensor_slices((val_df['text'].values, val_labels)).batch(64).prefetch(tf.data.AUTOTUNE)

In [ ]:
encode = keras.layers.TextVectorization()
encode.adapt(train_tensor.map(lambda text, label:text))

In [ ]:
#len(encode.get_vocabulary())

In [ ]:
model = keras.Sequential([
    encode,
    keras.layers.Embedding(len(encode.get_vocabulary()), 64, mask_zero=True),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(1)
])

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
#len(train_tensor)

In [ ]:
history = model.fit(
    train_tensor,
    epochs=10,
    validation_data=val_tensor
)

Epoch 1/10
66/66 [==============================] - 29s 257ms/step - loss: 0.6711 - accuracy: 0.8660 - val_loss: 0.6323 - val_accuracy: 0.8657
Epoch 2/10
66/66 [==============================] - 2s 33ms/step - loss: 0.5419 - accuracy: 0.8660 - val_loss: 0.4510 - val_accuracy: 0.8657
Epoch 3/10
66/66 [==============================] - 1s 20ms/step - loss: 0.3729 - accuracy: 0.8660 - val_loss: 0.2980 - val_accuracy: 0.8657
Epoch 4/10
66/66 [==============================] - 1s 19ms/step - loss: 0.2247 - accuracy: 0.8682 - val_loss: 0.1778 - val_accuracy: 0.8858
Epoch 5/10
66/66 [==============================] - 1s 19ms/step - loss: 0.1231 - accuracy: 0.9615 - val_loss: 0.0982 - val_accuracy: 0.9777
Epoch 6/10
66/66 [==============================] - 1s 19ms/step - loss: 0.0644 - accuracy: 0.9892 - val_loss: 0.0746 - val_accuracy: 0.9820
Epoch 7/10
66/66 [==============================] - 1s 19ms/step - loss: 0.0421 - accuracy: 0.9938 - val_loss: 0.0731 - val_accuracy: 0.9784
Epoch 8/10


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict([pred_text])
  if pred[0][0] >= 0.5:
      prediction = [pred[0][0], "spam"]
  else:
      prediction = [pred[0][0], "ham"]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 2s 2s/step
[-5.6000247, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 47ms/step
You passed the challenge. Great job!
